In [1]:
from pyspark.sql.functions import col ,coalesce

## Joins en PySpark con DataFrames y con SQL

Tipos de joins join (inner, outer, left_outer, right_outer, left_semi ,left_anti, right_semi, right_anti)
* inner
* left outer
* right outer
* full outer


Creamos 2 tablas
* **TablaA ('Messi',1),('Cristiano',2),('Neymar',3),('Benzema',4)**
* **TablaB  ('Messi',1),('Neymar',3),('Mbappe',5),('Rashford',6)**

In [3]:
### CREAMOS LAS TABLAS

	
datosA = [('Messi',1),('Cristiano',2),('Neymar',3),('Benzema',4)]
TablaA = spark.createDataFrame( data=datosA, schema =['nombreA','id'])
 
datosB = [('Messi',1),('Neymar',3),('Mbappe',5),('Rashford',6)]
TablaB = spark.createDataFrame( data=datosB, schema =['nombreB','id'])
 


#### Ver el contenido de las tablaA y tablaB

In [5]:
display(TablaA)

nombreA id Messi 1 Cristiano 2 Neymar 3 Benzema 4

In [6]:
display(TablaB)

nombreB id Messi 1 Neymar 3 Mbappe 5 Rashford 6

####  Creamos 2 alias para estas tablas

In [8]:
ta = TablaA.alias('ta')
tb = TablaB.alias('tb')

## INNER JOIN es la opción por defecto
### Todas las filas coincidentes que existe el id en tabla A y tabla B

Las siguientes instrucciones son equivalentes dado que how es el por defecto
* inner_join =  ta.join(  tb, ta.id == tb.id ,how='inner' )
* inner_join =  ta.join(  tb, ta.id == tb.id )

In [10]:
# inner_join =  ta.join(  tb, ta.id == tb.id ,how='inner' )

inner_join =  ta.join(  tb, ta.id == tb.id)
inner_join.show()

+-------+---+-------+---+
nombreA| id|nombreB| id|
+-------+---+-------+---+
 Messi| 1| Messi| 1|
 Neymar| 3| Neymar| 3|
+-------+---+-------+---+

## LEFT JOIN (LEFT OUTER)
### Todas las filas que están en la tabla A (Izquierda)

In [12]:
left_join = ta.join(tb, ta.id == tb.id,how='left') # Could also use 'left_outer'
left_join.show()

+---------+---+-------+----+
 nombreA| id|nombreB| id|
+---------+---+-------+----+
 Messi| 1| Messi| 1|
Cristiano| 2| null|null|
 Neymar| 3| Neymar| 3|
 Benzema| 4| null|null|
+---------+---+-------+----+

#### Filas que NO cruzan y tienen nombreB de tabla B  nulo

In [14]:
left_join = ta.join(tb, ta.id == tb.id,how='left') # Could also use 'left_outer'
left_join.filter(col('tb.id').isNull()).show()

+---------+---+-------+----+
 nombreA| id|nombreB| id|
+---------+---+-------+----+
Cristiano| 2| null|null|
 Benzema| 4| null|null|
+---------+---+-------+----+

### Filas que cruzan y tienen nombreB de tabla B no nulo

In [16]:
left_join = ta.join(tb, ta.id== tb.id,how='left') # Could also use 'left_outer'
left_join.filter(col('tb.id').isNotNull()).show()

+-------+---+-------+---+
nombreA| id|nombreB| id|
+-------+---+-------+---+
 Messi| 1| Messi| 1|
 Neymar| 3| Neymar| 3|
+-------+---+-------+---+

### Left Semi Join

**Esto se parece  a un inner join pero únicamente las columnas y valores del DataFrame izquierdo son seleccionados**

In [18]:
left_semi_join = ta.join(tb, ta.id == tb.id,how='left_semi')
left_semi_join.show()

+-------+---+
nombreA| id|
+-------+---+
 Messi| 1|
 Neymar| 3|
+-------+---+

### Left Anti Join

Este join es como hacer **A-B, y selecciona todas las filas de A que no están presentes en B. **

In [20]:
left_anti_join = ta.join(tb, ta.id == tb.id,how='left_anti')
left_anti_join.show()

+---------+---+
 nombreA| id|
+---------+---+
Cristiano| 2|
 Benzema| 4|
+---------+---+

# RIGHT JOIN (RIGHT OUTER)
#### Todas las filas que están en la tabla B de la derecha

In [22]:
right_join = ta.join(tb, ta.id == tb.id,how='right') # Could also use 'right_outer'
right_join.show()

+-------+----+--------+---+
nombreA| id| nombreB| id|
+-------+----+--------+---+
 Messi| 1| Messi| 1|
 Neymar| 3| Neymar| 3|
 null|null| Mbappe| 5|
 null|null|Rashford| 6|
+-------+----+--------+---+

##  FULL OUTER JOIN
### Todas las filas estén en A o estén en B

In [24]:
full_outer_join = ta.join(tb, ta.id == tb.id,how='full_outer') # Se puede usar tambien 'full' y 'outer'
full_outer_join.show()

+---------+----+--------+----+
 nombreA| id| nombreB| id|
+---------+----+--------+----+
 Messi| 1| Messi| 1|
Cristiano| 2| null|null|
 Neymar| 3| Neymar| 3|
 Benzema| 4| null|null|
 null|null| Mbappe| 5|
 null|null|Rashford| 6|
+---------+----+--------+----+

#### Otra forma similar de hacer lo mismo

In [26]:
full_outer = ta.join(tb, on=['id'], how='full_outer')  # full_outer, outer y full hacen lo mismo
full_outer.show()

+---+---------+--------+
 id| nombreA| nombreB|
+---+---------+--------+
 1| Messi| Messi|
 2|Cristiano| null|
 3| Neymar| Neymar|
 4| Benzema| null|
 5| null| Mbappe|
 6| null|Rashford|
+---+---------+--------+

In [27]:
full_outer.printSchema()

root
-- id: long (nullable = true)
-- nombreA: string (nullable = true)
-- nombreB: string (nullable = true)

### Coalesce para seleccionar la primera columna no nula de sus argumentos

In [29]:
full_outer= full_outer.withColumn('nombre_final', 
                                  coalesce(full_outer['nombreA'],full_outer['nombreB']) 
                                 )
full_outer.show()

+---+---------+--------+------------+
 id| nombreA| nombreB|nombre_final|
+---+---------+--------+------------+
 1| Messi| Messi| Messi|
 2|Cristiano| null| Cristiano|
 3| Neymar| Neymar| Neymar|
 4| Benzema| null| Benzema|
 5| null| Mbappe| Mbappe|
 6| null|Rashford| Rashford|
+---+---------+--------+------------+

# SQL JOINS
** Replicamos los joins con SQL**

Registramos los dataframes ta y tb como tablas tabla_a ytabla_b

In [31]:
ta.createOrReplaceTempView("tabla_a")
tb.createOrReplaceTempView("tabla_b")

In [32]:
### borrar una tabla
## spark.catalog.dropTempView("XXXX")

In [33]:
spark.sql("SHOW TABLES").show(truncate=False)

+--------+---------+-----------+
database|tableName|isTemporary|
+--------+---------+-----------+
 |tabla_a |true |
 |tabla_b |true |
+--------+---------+-----------+

### Repaso SELECT
Seleccion básica de todos los campos de la tabla A
* SELECT * selecciona todos los campos

In [35]:
spark.sql(" SELECT * FROM tabla_a ").show(truncate=False)

+---------+---+
nombreA |id |
+---------+---+
Messi |1 |
Cristiano|2 |
Neymar |3 |
Benzema |4 |
+---------+---+

**Seleccion básica de 1 campo nombreA de la tabla A**

In [37]:
spark.sql(" SELECT nombreA FROM tabla_a ").show(truncate=False)

+---------+
nombreA |
+---------+
Messi |
Cristiano|
Neymar |
Benzema |
+---------+

#### Seleccionamos todas las columnas de la tabla_a pero con id mayor que 2. 
 * **WHERE para filtrar**

In [39]:
spark.sql(" SELECT * FROM tabla_a WHERE id >2 ").show(truncate=False)

+-------+---+
nombreA|id |
+-------+---+
Neymar |3 |
Benzema|4 |
+-------+---+

#### Revisamos el número de filas de la tabla_a
 * **COUNT(*)  para contar todas las filas **
   * Asignamos el alias contador a la columna count(*)

In [41]:
spark.sql("  SELECT count(*) as contador   FROM     tabla_a   ").show(truncate=False)

+--------+
contador|
+--------+
4 |
+--------+

## INNER JOIN con SQL

In [43]:
spark.sql(" SELECT *                         \
            FROM tabla_a INNER JOIN tabla_b  \
             ON  tabla_a.id = tabla_b.id     \
         ").show(truncate=False)

+-------+---+-------+---+
nombreA|id |nombreB|id |
+-------+---+-------+---+
Messi |1 |Messi |1 |
Neymar |3 |Neymar |3 |
+-------+---+-------+---+

## LEFT JOIN con SQL

In [45]:
spark.sql(" SELECT *                         \
            FROM tabla_a LEFT JOIN tabla_b  \
             ON  tabla_a.id = tabla_b.id     \
         ").show(truncate=False)

+---------+---+-------+----+
nombreA |id |nombreB|id |
+---------+---+-------+----+
Messi |1 |Messi |1 |
Cristiano|2 |null |null|
Neymar |3 |Neymar |3 |
Benzema |4 |null |null|
+---------+---+-------+----+

#### Usamos alias para los nombres de tabla con AS y  seleccionamos 3 columnas
* tabla_a  as tabA
* tabla_b as tabB

In [47]:
spark.sql(" SELECT tabA.id, tabA.nombreA,    tabB.nombreB                  \
            FROM tabla_a as tabA                \
             LEFT JOIN tabla_b  as tabB            \
             ON  tabA.id = tabB.id                    \
         ").show(truncate=False)

+---+---------+-------+
id |nombreA |nombreB|
+---+---------+-------+
1 |Messi |Messi |
2 |Cristiano|null |
3 |Neymar |Neymar |
4 |Benzema |null |
+---+---------+-------+

## RIGHT JOIN con SQL

In [49]:
spark.sql(" SELECT *                         \
            FROM tabla_a RIGHT JOIN tabla_b  \
             ON  tabla_a.id = tabla_b.id     \
         ").show(truncate=False)

+-------+----+--------+---+
nombreA|id |nombreB |id |
+-------+----+--------+---+
Messi |1 |Messi |1 |
Neymar |3 |Neymar |3 |
null |null|Mbappe |5 |
null |null|Rashford|6 |
+-------+----+--------+---+

## FULL OUTER JOIN con SQL

In [51]:
spark.sql(" SELECT *                         \
            FROM tabla_a FULL OUTER JOIN tabla_b  \
             ON  tabla_a.id = tabla_b.id     \
         ").show(truncate=False)

+---------+----+--------+----+
nombreA |id |nombreB |id |
+---------+----+--------+----+
Messi |1 |Messi |1 |
Cristiano|2 |null |null|
Neymar |3 |Neymar |3 |
Benzema |4 |null |null|
null |null|Mbappe |5 |
null |null|Rashford|6 |
+---------+----+--------+----+